<a href="https://colab.research.google.com/github/v2sh1t/Colab-Stable-Diffusion/blob/main/SwarmUI-colab-notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#### PICK ONE: This node for temporary install
SWARMPATH = '/content/'

In [ ]:
#### PICK ONE: *OR* this node for install in google drive
from google.colab import drive
drive.mount('/content/drive')
SWARMPATH = '/content/drive/MyDrive'

# Colab breaks venvs, and doesn't save anything valid to drive, so just screw it do a global install of pip reqs if we used a comfy backend in the drive
!if [[ -f "$SWARMPATH/SwarmUI/dlbackend/ComfyUI/requirements.txt" ]]; then rm -rf $SWARMPATH/SwarmUI/dlbackend/ComfyUI/venv/; pip install -r $SWARMPATH/SwarmUI/dlbackend/ComfyUI/requirements.txt; fi

In [2]:
# Install dotnet dependencies
!wget https://dot.net/v1/dotnet-install.sh -O dotnet-install.sh
!chmod +x dotnet-install.sh
!./dotnet-install.sh --channel 8.0

# Install Clouldflared
!wget https://github.com/cloudflare/cloudflared/releases/download/2024.8.2/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

# use the chosen path
import os
os.environ['SWARMPATH'] = SWARMPATH
%cd $SWARMPATH

# Colab breaks venv, so, tell swarm to not make a venv
os.environ['SWARM_NO_VENV'] = 'true'

# Download SwarmUI
url = "https://github.com/mcmonkeyprojects/SwarmUI"
!git clone $url

--2025-01-01 13:29:13--  https://dot.net/v1/dotnet-install.sh
Resolving dot.net (dot.net)... 20.76.201.171, 20.112.250.133, 20.236.44.162, ...
Connecting to dot.net (dot.net)|20.76.201.171|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dotnet.microsoft.com/download/dotnet/scripts/v1/dotnet-install.sh [following]
--2025-01-01 13:29:14--  https://dotnet.microsoft.com/download/dotnet/scripts/v1/dotnet-install.sh
Resolving dotnet.microsoft.com (dotnet.microsoft.com)... 13.107.246.73, 2620:1ec:bdf::73
Connecting to dotnet.microsoft.com (dotnet.microsoft.com)|13.107.246.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Cookie coming from dotnet.microsoft.com attempted to set domain to dotnetwebsite.azurewebsites.net
Cookie coming from dotnet.microsoft.com attempted to set domain to dotnetwebsite.azurewebsites.net
Length: 65888 (64K) [application/x-sh,application/octet-stream]
Saving to: ‘dotnet-install.sh’

dotnet-install

In [5]:
# @title Download Model
# @markdown Select which models to download:

# @markdown 1: Download FP16 FLUX Dev - For 32GB+ GPUs - Best quality

# @markdown 2: Download FP8 FLUX Dev - For 24 GB and below GPUs - Good quality - Scaled

# @markdown 3: Download FP16 Stable Diffusion 3.5 Large - Best Quality

# @markdown 4: Download FP8 Scaled Stable Diffusion 3.5 Large - Good Quality - Recommended for Low VRAM GPUs

# @markdown 5: Download FP16 T5 - Recommended

# @markdown 6: Download FP8 T5 - Further reduces VRAM usage

# @markdown 7: Download Stable Diffusion 3.5 Medium

# @markdown 8: Download FP8 Scaled Stable Diffusion 3.5 Medium

# @markdown 9: Download best Stable Diffusion 1.5 and SDXL Models

# @markdown 10: Download FP8 FLUX Dev - For 24 GB and below GPUs - Good quality - Original

# @markdown 11: Download Best Man and Woman Face Segment Models - Yolo Based - Auto Mask to Inpaint

# @markdown 12: Download SOTA Text-to-Video Model Genmo Mochi 1

# @markdown 13: Download FLUX Redux (Style Model)

# @markdown 14: Download FLUX Depth (Full Checkpoint - 23.8 GB)

# @markdown 15: Download FLUX Canny (Full Checkpoint - 23.8 GB)

# @markdown 16: Download FLUX Canny LoRA (1.24 GB)

# @markdown 17: Download FLUX Depth LoRA (1.24 GB)

# @markdown 18: Download FLUX Fill (Full Checkpoint - 23.8 GB) - Inpainting and Outpainting

# @markdown 19: Download Newest Better Clip-L Model - Improves Quality at Everything I Tested (CLIP-SAE-ViT-L-14)

# @markdown 20: Download Very Best Deterministic Upscale Models (They are From OpenModelDB) - Request New Ones Uf You Need

# @markdown 21: Download FLUX ControlNet Union Pro Model

# @markdown 0: Exit
choices = "2,6,11,13,14,15,18,19,20" # @param {"type":"string","placeholder":"0"}

import sys
import subprocess
import os
import platform
import shutil

def install_huggingface_hub():
  subprocess.check_call([sys.executable, "-m", "pip", "install", "huggingface_hub>=0.25.2"])

def install_hf_transfer():
  subprocess.check_call([sys.executable, "-m", "pip", "install", "hf_transfer>=0.1.8"])

# Check for huggingface_hub
try:
  from huggingface_hub import snapshot_download
except ImportError:
  print("huggingface_hub is not installed. Installing now...")
  install_huggingface_hub()
  from huggingface_hub import snapshot_download

# Check for hf_transfer
try:
  import hf_transfer
except ImportError:
  print("hf_transfer is not installed. Installing now...")
  install_hf_transfer()
  import hf_transfer

system = platform.system()
if system == "Windows":
  base_Path = "SwarmUI\\Models"
else:  # Linux/Unix systems
  if os.path.exists("/home/Ubuntu/apps/StableSwarmUI"):
    base_Path = "/home/Ubuntu/apps/StableSwarmUI/Models"
  elif os.path.exists("/workspace/SwarmUI"):
    base_Path = "/workspace/SwarmUI/Models"
  else:
    base_Path = "SwarmUI/Models"

def ensure_directories_exist():
  # Create the base Models directory and all subdirectories
  directories = [
    base_Path,
    os.path.join(base_Path, "vae"),
    os.path.join(base_Path, "diffusion_models"),
    os.path.join(base_Path, "Stable-Diffusion"),
    os.path.join(base_Path, "clip"),
    os.path.join(base_Path, "yolov8"),
    os.path.join(base_Path, "style_models"),
    os.path.join(base_Path, "Lora"),
    os.path.join(base_Path, "upscale_models")
  ]

  for directory in directories:
    try:
      os.makedirs(directory, exist_ok=True)
      print(f"Created or verified directory: {directory}")
    except Exception as e:
      print(f"Error creating directory {directory}: {str(e)}")

def download_models(options):
  # Create all necessary directories first
  ensure_directories_exist()

  # Always download the VAE
  snapshot_download(
    repo_id="black-forest-labs/FLUX.1-schnell",
    allow_patterns=["ae.safetensors"],
    local_dir=f"{base_Path}/vae"
  )
  # Always download the FLUX ControlNet Model


  if 1 in options:  # FP16 FLUX Dev
    snapshot_download(
      repo_id="OwlMaster/FLUX_LoRA_Train",
      allow_patterns="flux1-dev.safetensors",
      local_dir=f"{base_Path}/diffusion_models"
    )
  if 2 in options:  # FP8 FLUX Dev Scaled
    snapshot_download(
      repo_id="OwlMaster/SD3New",
      allow_patterns="flux_dev_fp8_scaled_diffusion_model.safetensors",
      local_dir=f"{base_Path}/diffusion_models"
    )
  if 3 in options:  # FP16 SD3.5
    snapshot_download(
      repo_id="OwlMaster/SD3New",
      allow_patterns="sd3.5_large.safetensors",
      local_dir=f"{base_Path}/Stable-Diffusion"
    )
  if 4 in options:  # FP8 SD3.5 Scaled
    snapshot_download(
      repo_id="OwlMaster/SD3New",
      allow_patterns="sd3.5_large_fp8_scaled.safetensors",
      local_dir=f"{base_Path}/Stable-Diffusion"
    )
  if 5 in options:  # FP16 T5
    snapshot_download(
      repo_id="OwlMaster/SD3New",
      allow_patterns=["clip_l.safetensors", "t5xxl_fp16.safetensors","clip_g.safetensors"],
      local_dir=f"{base_Path}/clip"
    )
  if 6 in options:  # FP8 T5
    snapshot_download(
      repo_id="OwlMaster/SD3New",
      allow_patterns=["clip_l.safetensors", "t5xxl_fp8_e4m3fn_scaled.safetensors","clip_g.safetensors"],
      local_dir=f"{base_Path}/clip"
    )
  if 7 in options:  # SD3.5 Medium
    snapshot_download(
      repo_id="OwlMaster/SD3New",
      allow_patterns="sd3.5_medium.safetensors",
      local_dir=f"{base_Path}/Stable-Diffusion"
    )
  if 8 in options:  # FP8 Scaled SD3.5 Medium
    snapshot_download(
      repo_id="OwlMaster/SD3New",
      allow_patterns="sd3.5_medium_incl_clips_t5xxlfp8scaled.safetensors",
      local_dir=f"{base_Path}/Stable-Diffusion"
    )
  if 9 in options:  # SD1.5 and SDXL models
    snapshot_download(
        repo_id="MonsterMMORPG/Best_SD15_SDXL_Models",
        local_dir=f"{base_Path}/Stable-Diffusion"
    )
  if 10 in options:  # FP8 FLUX Dev
    snapshot_download(
      repo_id="Kijai/flux-fp8",
      allow_patterns="flux1-dev-fp8.safetensors",
      local_dir=f"{base_Path}/diffusion_models"
    )
  if 11 in options:  # Face Segment Models
    snapshot_download(
      repo_id="MonsterMMORPG/FaceSegments",
      allow_patterns=["man_face.pt", "woman_face.pt","face_yolov9c.pt"],
      local_dir=f"{base_Path}/yolov8"
    )
  if 12 in options:  # mochi
    snapshot_download(
      repo_id="Comfy-Org/mochi_preview_repackaged",
      allow_patterns=["all_in_one/mochi_preview_fp8_scaled.safetensors"],
      local_dir=f"{base_Path}/Stable-Diffusion"
    )
  if 13 in options:  # FLUX Redux (Style Model)
    snapshot_download(
      repo_id="OwlMaster/SD3New",
      allow_patterns="flux1-redux-dev.safetensors",
      local_dir=f"{base_Path}/style_models"
    )
  if 14 in options:  # FLUX Depth (Full Checkpoint)
    snapshot_download(
      repo_id="OwlMaster/SD3New",
      allow_patterns="flux1-depth-dev.safetensors",
      local_dir=f"{base_Path}/diffusion_models"
    )
    print("Downloaded FLUX Depth (Full Checkpoint - 23.8 GB)")
  if 15 in options:  # FLUX Canny (Full Checkpoint)
    snapshot_download(
      repo_id="OwlMaster/SD3New",
      allow_patterns="flux1-canny-dev.safetensors",
      local_dir=f"{base_Path}/diffusion_models"
    )
    print("Downloaded FLUX Canny (Full Checkpoint - 23.8 GB)")
  if 16 in options:  # FLUX Canny LoRA
    snapshot_download(
      repo_id="OwlMaster/SD3New",
      allow_patterns="flux1-canny-dev-lora.safetensors",
      local_dir=f"{base_Path}/Lora"
    )
    print("Downloaded FLUX Canny LoRA (1.24 GB)")
  if 17 in options:  # FLUX Depth LoRA
    snapshot_download(
      repo_id="OwlMaster/SD3New",
      allow_patterns="flux1-depth-dev-lora.safetensors",
      local_dir=f"{base_Path}/Lora"
    )
    print("Downloaded FLUX Depth LoRA (1.24 GB)")
  if 18 in options:  # FLUX Fill (Full Checkpoint)
    snapshot_download(
      repo_id="OwlMaster/SD3New",
      allow_patterns="flux1-fill-dev.safetensors",
      local_dir=f"{base_Path}/diffusion_models"
    )
    print("Downloaded FLUX Fill (Full Checkpoint - 23.8 GB) - Inpainting Outpainting")
  if 19 in options:  # Newest Better Clip-L Model
    clip_l_path = os.path.join(base_Path, "clip", "clip_l.safetensors")
    if os.path.exists(clip_l_path):
      os.remove(clip_l_path)
      print("Removed existing clip_l.safetensors")
    snapshot_download(
      repo_id="OwlMaster/zer0int-CLIP-SAE-ViT-L-14",
      allow_patterns="clip_l.safetensors",
      local_dir=f"{base_Path}/clip"
    )
    print("Downloaded Newest Better Clip-L Model (CLIP-SAE-ViT-L-14)")
  if 20 in options:  # Best Upscaler Models
    snapshot_download(
      repo_id="OwlMaster/best_upscaler_models",
      local_dir=f"{base_Path}/upscale_models"
    )
    print("Downloaded Very Best Deterministic Upscale Models")
  if 21 in options:
    snapshot_download(
      repo_id="Shakker-Labs/FLUX.1-dev-ControlNet-Union-Pro",
      allow_patterns=["diffusion_pytorch_model.safetensors"],
      local_dir=f"{base_Path}/controlnet/Shakker-Labs/FLUX.1-dev-ControlNet-Union-Pro"
    )
    print("Downloaded FLUX ControlNet Union Pro Model")

def parse_input(choices):
  if ',' in choices:
    return [int(x.strip()) for x in choices.split(',')]
  elif '-' in choices:
    return [int(x.strip()) for x in choices.split('-')]
  else:
    return [int(choices.strip())]


if __name__ == "__main__":
  if len(choices) > 1:
    try:
      selected_options = parse_input(choices)
      download_models(selected_options)
    except ValueError:
      print("Invalid input. Please enter comma or dash separated numbers.")
  else:
    while True:
      print("Select which models to download:")
      print("1: Download FP16 FLUX Dev - For 32GB+ GPUs - Best quality")
      print("2: Download FP8 FLUX Dev - For 24 GB and below GPUs - Good quality - Scaled")
      print("3: Download FP16 Stable Diffusion 3.5 Large - Best Quality")
      print("4: Download FP8 Scaled Stable Diffusion 3.5 Large - Good Quality - Recommended for Low VRAM GPUs")
      print("5: Download FP16 T5 - Recommended")
      print("6: Download FP8 T5 - Further reduces VRAM usage")
      print("7: Download Stable Diffusion 3.5 Medium")
      print("8: Download FP8 Scaled Stable Diffusion 3.5 Medium")
      print("9: Download best Stable Diffusion 1.5 and SDXL Models")
      print("10: Download FP8 FLUX Dev - For 24 GB and below GPUs - Good quality - Original")
      print("11: Download Best Man and Woman Face Segment Models - Yolo Based - Auto Mask to Inpaint")
      print("12: Download SOTA Text-to-Video Model Genmo Mochi 1")
      print("13: Download FLUX Redux (Style Model)")
      print("14: Download FLUX Depth (Full Checkpoint - 23.8 GB)")
      print("15: Download FLUX Canny (Full Checkpoint - 23.8 GB)")
      print("16: Download FLUX Canny LoRA (1.24 GB)")
      print("17: Download FLUX Depth LoRA (1.24 GB)")
      print("18: Download FLUX Fill (Full Checkpoint - 23.8 GB) - Inpainting and Outpainting")
      print("19: Download Newest Better Clip-L Model - Improves Quality at Everything I Tested (CLIP-SAE-ViT-L-14)")
      print("20: Download Very Best Deterministic Upscale Models (They are From OpenModelDB) - Request New Ones Uf You Need")
      print("21: Download FLUX ControlNet Union Pro Model")
      print("0: Exit")

      choices = input("Enter your choices (comma or dash separated, e.g., 1,3,5 or 1-3-5): ")
      if choices == "0":
          break

      try:
          selected_options = parse_input(choices)
          download_models(selected_options)
      except ValueError:
          print("Invalid input. Please enter comma or dash separated numbers.")


hf_transfer is not installed. Installing now...
Created or verified directory: SwarmUI/Models
Created or verified directory: SwarmUI/Models/vae
Created or verified directory: SwarmUI/Models/diffusion_models
Created or verified directory: SwarmUI/Models/Stable-Diffusion
Created or verified directory: SwarmUI/Models/clip
Created or verified directory: SwarmUI/Models/yolov8
Created or verified directory: SwarmUI/Models/style_models
Created or verified directory: SwarmUI/Models/Lora
Created or verified directory: SwarmUI/Models/upscale_models


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

ae.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

...

4xRealWebPhoto_v4_drct-l.pth:   0%|          | 0.00/243M [00:00<?, ?B/s]

4xNomos2_hq_dat2.pth:   0%|          | 0.00/140M [00:00<?, ?B/s]

4x_RealisticRescaler_100000_G.pth:   0%|          | 0.00/134M [00:00<?, ?B/s]

RealESRGAN_x4plus.pth:   0%|          | 0.00/67.0M [00:00<?, ?B/s]

Downloaded Very Best Deterministic Upscale Models


In [ ]:
# Alright, launch it! Watch the output for a Cloudflare URL
%cd $SWARMPATH/SwarmUI/
!git pull

!bash ./launch-linux.sh --launch_mode none --cloudflared-path cloudflared

/content/SwarmUI
Already up to date.
cat: src/bin/last_build: No such file or directory



mv: cannot stat './src/bin/live_release': No such file or directory
=========
Welcome to .NET 8.0!
---------------------
SDK Version: 8.0.404

----------------
Installed an ASP.NET Core HTTPS development certificate.
To trust the certificate, view the instructions: https://aka.ms/dotnet-https-linux

----------------
Write your first app: https://aka.ms/dotnet-hello-world
Find out what's new: https://aka.ms/dotnet-whats-new
Explore documentation: https://aka.ms/dotnet-docs
Report issues and find source on GitHub: https://github.com/dotnet/core
Use 'dotnet --help' to see available commands or visit: https://aka.ms/dotnet-cli
--------------------------------------------------------------------------------------
  Determining projects to restore...
  Restored /content/SwarmUI/src/SwarmUI.csproj (in 7.56 sec).
  SwarmUI -> /content/SwarmUI/src/bin/live_release/SwarmUI.dll

Build succeeded.
  